# 局部线性嵌入(Lcally Linear Embedding, LLE)

## 符号定义

|符号|含义|
|:-:|:-:|
|$\bm{x}$|样本点|
|$X_i$|样本点$\bm{x_i}$的k近邻点集合|
|$N$|总样本数|
|$N_i$|样本点$\bm{x_i}$的k近邻点集合的样本总数|
|$\bm{X_i^{(j)}}$|样本点$\bm{x_i}$的k近邻点集合中的第j个样本点|
|$\bm{w}$|线性组合的权重向量|
|$\bm{y}$|样本点降维后坐标|

## 概念

LLE属于流形学习。流形学习假设所有数据点分布在嵌入于外维欧式空间的一个潜在的流形体上。LLE的核心思想是：流形在局部领域上是线性的，即任意一个点都可以由其周围的点在最小二乘意义下进行线性表示；降维后，任意一个点依然可以由其周围的点在最小二乘意义下进行线性表示，并且线性表示的系数不变。

## 原理

由于LLE假设局部是线性的，因此对于数据点$\bm{x}$，其可以由其k-近邻数据点的线性组合进行表示

$$
\begin{equation}
    \bm{x_i} = \sum_{j=1}^{N_i}w_j\bm{x_j}
\end{equation}
$$

由于LLE假设降维后依然可以用相同的系数对数据点$\bm{x}$进行表示，所以有

$$
\begin{equation}
    \bm{y_i} = \sum_{j=1}^{N_i}w_j\bm{y_j}
\end{equation}
$$

由上分析可以发现LLE有三步：
* 第一步为求得任意数据点$\bm{x}$的k-近邻数据点集合；
* 第二步为求得任意数据点$\bm{x}$的k-近邻数据点线性表示，并得到对应的权重向量$\bm{w}$；
* 第三步为求得符合权重向量$\bm{w}$的低维表示

## 推导

### 线性组合权重系数求解

对于线性表示的权重系数有如下的损失函数

$$
\begin{equation}
    \mathcal{L} = \sum_{i=1}^N||\bm{x_i}-\sum_{j=1}^{N_i}w_{ij}\bm{X_i^{(j)}}||_2^2, s.t. \sum_{j=1}^{N_i}w_{ij}=1
\end{equation}
$$

最佳的权重系数显然是上述的损失最小，即$\arg\min\limits_{W}\mathcal{L}$，其中$W$为由所有样本点的权重系数组成的矩阵

有
$$
\begin{equation}
   \begin{split}
   \arg\min\limits_{W}\mathcal{L} &=  \arg\min\limits_{W}\sum_{i=1}^N||\bm{x_i}-\sum_{j=1}^{N_i}w_{ij}\bm{X_i^{(j)}}||_2^2 \\
   &= \arg\min\limits_{W}\sum_{i=1}^N||\sum_{j=1}^{N_i}(\bm{x_i}-\bm{X_i^{(j)}})w_{ij}||_2^2 \\
   &= \arg\min\limits_{W}\sum_{i=1}^N||([\underbrace{\bm{x_i},\bm{x_i}, \cdots, \bm{x_i}}_{N_i}]-X_i)\bm{w_i}||_2^2 \\
   &= \arg\min\limits_{W}\sum_{i=1}^N\bm{w_i}^T([\underbrace{\bm{x_i},\bm{x_i}, \cdots, \bm{x_i}}_{N_i}]-X_i)^T([\underbrace{\bm{x_i},\bm{x_i}, \cdots, \bm{x_i}}_{N_i}]-X_i)\bm{w_i}
   \end{split}, s.t. \sum_{j=1}^{N_i}w_{ij}=\bm{w_i}^T[1, 1, \cdots, 1]_k^T = 1
\end{equation}
$$

即求解如下存在约束的优化问题
$$
\begin{equation}
    \left\{
    \begin{split}
    & \min\mathcal{L}, \\
    & s.t. \bm{w_i}^T[1, 1, \cdots, 1]_k^T - 1 = 0
    \end{split}
    \right . 
\end{equation}
$$

考虑运用拉格朗日乘子法
即
$$
\begin{equation}
    \left\{
    \begin{split}
    &\frac{\partial{\mathcal{L}}}{\partial{\bm{w_i}}}-\lambda\frac{\partial{(\bm{w_i}^T[1, 1, \cdots, 1]_k^T - 1)}}{\partial{\bm{w_i}}} = \bm{0} \\
    & \bm{w_i}^T[1, 1, \cdots, 1]_k^T - 1 = 0
    \end{split}
    \right .
\end{equation}
$$

记$S_i=([\underbrace{\bm{x_i},\bm{x_i}, \cdots, \bm{x_i}}_{N_i}]-X_i)^T([\underbrace{\bm{x_i},\bm{x_i}, \cdots, \bm{x_i}}_{N_i}]-X_i)$

同时记$\bm{1_k} = [1, 1, \cdots, 1]_k^T$

得到
$$
\begin{equation}
    \begin{split}
        &2S_i\bm{w_i} - \lambda\bm{1_k} = 0\\
        \Rightarrow &\bm{w_i} = \frac{1}{2}S_i^{-1}\lambda\bm{1_k}
    \end{split}
\end{equation}
$$

又因为$\bm{w_i}^T\bm{1_k} - 1 = 0$

可以得到
$$
\begin{equation}
    \begin{split}
        & \bm{1_k}^T \frac{1}{2}S_k^{-1}\lambda\bm{1_k} = 1 \\
        \Rightarrow &\lambda = \frac{1}{\bm{1_k}^T \frac{1}{2}S_i^{-1}\bm{1_k}}
    \end{split}
\end{equation}
$$

所以得到

$$
\begin{equation}
    \bm{w_i} = \frac{S_i^{-1}\bm{1_k}}{\bm{1_k}^TS_i^{-1}\bm{1_k}}, i=1, 2, \cdots, N
\end{equation}
$$

应用上式即可求得所有数据点由k-近邻数据点线性表示的权重向量

## 低维表示求解

考虑到低维表示下，依然需要满足上一步得到的线性组合结果，因此有如下的损失函数

$$
\begin{equation}
    \mathcal{L_2} = \sum_{i=1}^N||\bm{y_i}-\sum_{j=1}^{N_i}w_{ij}\bm{Y_i^{(j)}}||_2^2, s.t. \sum_{i=1}^N\bm{y_i}=0, \sum_{i=1}^N\bm{y_i}\bm{y_i}^T=N\bm{I_{d\times d}}
\end{equation}
$$

上式不同于第二步。第二步是线性组合的权重未知，但是数据点已知。但是在第三步是线性组合的权重已知，而降维后的坐标未知。

定义如下的矩阵

$$
\begin{equation}
    V = [v_{ji}]_{N\times N}, 
    v_{ji} = 
    \left\{
    \begin{split}
    & w_{ij}, 数据点j是数据点i的近邻点 \\
    & 0, else
    \end{split}
    \right . 
\end{equation}
$$

因此有下式

$$
\begin{equation}
    \sum_{j=1}^{N_i}w_{ij}\bm{Y_i^{(j)}} = \sum_{j=1}^Nv_{ji}\bm{Y_i^{(j)}} = Y\bm{V_i}
\end{equation}
$$

其中$\bm{V_i}$是矩阵$V$的第$i$列

$$
\begin{equation}
    \begin{split}
    \mathcal{L_2} 
    &= \sum_{i=1}^N||\bm{y_i} - Y\bm{V_i}||_2^2 \\
    &= \sum_{i=1}^N||Y(\bm{I_i} - \bm{V_i})||_2^2
    \end{split}
\end{equation}
$$

其中$\bm{I_i}$是单位矩阵的第$i$列，上式中的$Y(\bm{I_i} - \bm{V_i})$可以被视为一个$N\times N$矩阵的第$i$列

由于对于任意矩阵$A=[\bm{a_1}, \bm{a_2}, \cdots, \bm{a_{N}}]$，有
$$
\begin{equation}
    \sum_{i=1}^N||\bm{a_i}||_2^2 = \sum_{i=1}^N\bm{a_i}^T\bm{a_i} = tr(AA^T)
\end{equation}
$$

因此可以得到
$$
\begin{equation}
    \mathcal{L_2}=tr(Y(I-V)(I-V)^TY^T)
\end{equation}
$$

令上式中的$(I-V)(I-V)^T=M=M^T$
有
$$
\begin{equation}
    \mathcal{L_2}=tr(YMY^T)
\end{equation}
$$


第三步为如下的优化问题

$$
\begin{equation}
    \left\{
    \begin{split}
    & \min\limits_{Y} \mathcal{L_2} \\
    & YY^T = N\bm{I_{d\times d}}
    \end{split}
    \right .
\end{equation}
$$

同样的，使用拉格朗日乘子法可得
$$
\begin{equation}
    \left\{
    \begin{split}
    &\frac{\partial{\mathcal{L_2}}}{\partial{Y}}-\lambda\frac{\partial({YY^T-N\bm{I_{d\times d}}})}{\partial{Y}} = 0 \\
    & YY^T = N\bm{I_{d\times d}}
    \end{split}
    \right.
\end{equation}
$$

由$\frac{\partial{\mathcal{L_2}}}{\partial{Y}}-\lambda\frac{\partial({YY^T-N\bm{I_{d\times d}}})}{\partial{Y}} = 0$可得

$$
\begin{equation}
    \begin{split}
    & YM+YM^T-2\lambda Y=0 \\
    \Rightarrow &YM^T=\lambda Y \\
    \Rightarrow &MY^T=\lambda Y^T
    \end{split}
\end{equation}
$$

显然，$Y^T$为矩阵$M$特征向量构成的矩阵

由此可得
$$
\begin{equation}
    \mathcal{L_2} = tr(YMY^T) = tr(\lambda YY^T)=tr(\lambda N\bm{I}_{d\times d})=\lambda Nd
\end{equation}
$$

因此为了使得损失$\mathcal{L_2}$尽可能小，应当选择矩阵$M$前d个最小特征值对应的特征向量。

又因为$M$为对称非负定矩阵，且$(I-V)\bm{1_N}=0$，因此$M$的最小特征值为0。在实际使用时，通常排除0这一最小特征值对应的特征向量，而选用其后的d个特征值对应的特征向量